# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Ishan Jain, ikj

Member 2: Name, UNI

Member 3: Name, UNI

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [3]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.parameters['W'] = []
        self.parameters['b'] = []
        self.num_layers = layer_dimensions
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        self.W = []
        self.b = []
        
        print(self.num_layers)
        print(layer_dimensions)
        
        for i in range(1, len(self.num_layers)):
            self.W.append(np.random.rand(self.num_layers[i-1], self.num_layers[i]))
            self.b.append(np.ones((self.num_layers[i], 1)))
        
        # init parameters
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        
        Z = np.matmul(W.T, A)
        Z = np.add(Z, b)
        cache = 0 
        return Z
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        
        A_relu = self.relu(A)
        return A_relu


    def relu(self, X):
        print(X.shape)
        A = np.maximum(0,X)
        return A

            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """

        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        
        cache = {}
        cache['A'] = []
        cache['Z'] = []
        
        
        print(X.shape)
        
        for i in range(1, len(self.num_layers)):
            Z = self.affineForward(X, self.W[i-1], self.b[i-1])
            print(Z.shape)
            X = self.activationForward(Z)
            print(X.shape)
            cache['A'].append(X)
            cache['Z'].append(Z)
           
        AL = X
        

        return AL, cache
    
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        temp1,temp2 = AL.shape
        cost = 0
        for i in range(temp2):
            cost = cost - np.log(np.exp(AL[y[i],i])/np.sum(np.exp(AL[:,i]),axis=0))
        cost = cost/temp2
        #if self.reg_lambda > 0:
            # add regularization
            
       
        
        # gradient of cost
        dAL = np.zeros((temp1,temp2))
        for i in range(temp2):
            for j in range(temp1):
                dAL[j,i] = np.exp(AL[j,i])/np.sum(np.exp(AL[:,i]),axis=0)
            dAL[y[i],i] = np.exp(AL[y[i],i])/np.sum(np.exp(AL[:,i]),axis=0) - 1
        
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        m = dA_prev.shape[1]
        dW = 1/m*np.matmul(dA_prev,cache['A'].T)
        db = np.mean(dA_prev,axis=0)
        dA = dA_prev

        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        relu_derivative(cache['Z'], cache['Z'])

        
    def relu_derivative(self, dx, cached_x):
        dx = np.ones(dx.shape)
        dx[cached_x < 0] = 0
        return dx

    def dropout_backward(self, dA, cache):

        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {'dW': [0 for _ in range(len(self.parameters['W']))], 'db': [0 for _ in range(len(self.parameters['b']))]}
        
        dA = np.copy(dAL)
        
        for i in range(len(self.parameters['W'])-1, -1, -1):
            _, dW, db = self.affineBackward(dA, cache[i])
            gradients['dW'][i] = dW
            gradients['db'][i] = db
            dA = np.matmul(self.parameters['W'][i].T,dA)
            dA = self.activationBackward(dA, cache[i])
            
            
            #if self.drop_prob > 0:
                #call dropout_backward
           
            
        #if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        ww = np.array(self.W)
        bb = np.array(self.b)
        print("Weights:")
        print(ww.shape,bb.shape)
        for i in range(len(self.W)):
            self.W[i] = np.subtract(self.W[i], alpha * gradients['dW'][i])
            self.b[i] = np.subtract(self.b[i], alpha * gradients['db'][i])
            
    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        
        # TODO improve array concatenation time
        
        #X, y = np.random.shuffle(X, y)
               
        X_batch = []
        y_batch = []
        for i in range(0, X.shape[1], batch_size):
            X_mini_batch = X[:, i: max(i + batch_size, X.shape[1])]
            y_mini_batch = y[i: max(i + batch_size, y.shape[0])]
            assert X_mini_batch.shape[1] == y_mini_batch.shape[0]
            X_batch.append(X_mini_batch)
            y_batch.append(y_mini_batch)            
                
                
                
    
        return tuple(X_batch), tuple(y_batch)

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        #assert X.shape[1] == y.shape[0]
        X_batch, y_batch = self.get_batch(X, y, batch_size)
        
        
        for i in range(0, iters):
            # get minibatch
                
                for j in range(len(X_batch)):
                    X_mini_batch = X_batch[j]
                    y_mini_batch = y_batch[j]
                    assert X_mini_batch.shape[1] == y_mini_batch.shape[0]          
                    
                    # forward prop
                    AL, cache = self.forwardPropagation(X_mini_batch)                

                    # compute loss
                    cost, dAL = self.costFunction(AL, y_mini_batch)

                    # compute gradients
                    gradients = self.backPropagation(dAL, y_mini_batch, cache)

                    # update weights and biases based on gradient
                    self.updateParameters(gradients, alpha)
                    

                #if i % print_every == 0:
                    # print cost, train and validation set accuracies
                    #TODO
                    #print "cost"
                    
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        y_pred = 0
        return y_pred

    

In [4]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [5]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [6]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [7]:
layer_dimensions = [X_train.shape[0], 20, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train) # iters, alpha, batch_size, print_every)

[3072, 20, 10]
[3072, 20, 10]
(3072, 50000)
(20, 50000)

/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:123: RuntimeWarning: overflow encountered in exp
/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in double_scalars
/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:134: RuntimeWarning: overflow encountered in exp



(20, 50000)
(20, 50000)
(10, 50000)
(10, 50000)
(10, 50000)
Weights:
(2,) (2,)


/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in double_scalars
/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:135: RuntimeWarning: overflow encountered in exp
/Users/Siddhant/miniconda2/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in double_scalars


IndexError: list index out of range

In [ ]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

In [24]:
ss = np.array([1,2,3])
print(ss.reshape(3,1).shape)
a,b = ss.reshape(3,1).shape
print(a,b)

(3, 1)
3 1
